## Submit spike sorting
This script generates cluster jobs for spikesorting (Churchland lab pipeline)

In [27]:
import labdatatools as ldt
from pathlib import Path
import numpy as np
import subprocess

SUBJECT = 'MM012'
N_PROBES = 3 # the number of probes in the recording, script will check that each probe was sorted
GPU = 'RTX2080Ti'
CLUSTER = 'Hodgkin' #generate command for Hoffman or Hodgkin

In [28]:
files = ldt.rclone_list_files(SUBJECT).dropna()
ephys_sessions = files[files.datatype.str.contains('ephys_')]
ksfolders = files[files.datatype.str.contains('kilosort2.5') & files.filename.str.contains('spike_times.npy')]
ephys_sessions = np.unique([f'{s.session}' for i,s in ephys_sessions.iterrows()])

In [29]:
sorted_sessions, unsorted_sessions = [],[]
for sess in ephys_sessions:
    fold = ksfolders.session.str.contains(sess)
    if sum(fold) == N_PROBES:
        sorted_sessions.append(sess)
    else:
        unsorted_sessions.append(sess)

print(f'There are {len(sorted_sessions)} sorted sessions')
print(f'There are {len(unsorted_sessions)} unsorted sessions')

There are 1 sorted sessions
There are 4 unsorted sessions


In [30]:
for s in unsorted_sessions:
    if CLUSTER == 'Hoffman':
        cmd = f'labdata submit spks -a {SUBJECT} -s {s} --module matlab --conda-env spks --queue gpu,{GPU},h_rt=6:00:00,cuda=1 --delete-session --overwrite -- -t \$TMPDIR' # for hoffman
    elif CLUSTER == 'Hodgkin':
        cmd = f'labdata submit spks -a {SUBJECT} -s {s} -g 1 -n 8 -m 50 --delete-session --overwrite -- -t /scratch' # for hodgkin
    print(cmd)

labdata submit spks -a MM012 -s 20231113_180427 -g 1 -n 8 -m 50 --delete-session --overwrite -- -t /scratch
labdata submit spks -a MM012 -s 20231114_163339 -g 1 -n 8 -m 50 --delete-session --overwrite -- -t /scratch
labdata submit spks -a MM012 -s 20231201_171654 -g 1 -n 8 -m 50 --delete-session --overwrite -- -t /scratch
labdata submit spks -a MM012 -s 20231205_174217 -g 1 -n 8 -m 50 --delete-session --overwrite -- -t /scratch
